# Sensitivity Analysis Made Easy with the EMA Workbench: Complete demo

This is the main notebook of the workshop on *sensitivity analysis* (SA) at the Social Simulation Festival 2021. Here we make a **more comperhansive demonstration** of how to do Variance-based SA also know as [Sobol SA](https://en.wikipedia.org/wiki/Variance-based_sensitivity_analysis) on a relatively simple model [virus on network](https://ccl.northwestern.edu/netlogo/models/VirusonaNetwork). The idea is that you reuse (read copy-paste) this code your own model. Therefore, we tried to keep simple and avoid extensive side steps from.

This notebook is tuned to be run on [Google Colab](https://colab.research.google.com/) and has a couple of extra lines of code. If you want to use it on your local machine please use `sa_demo_local_machine.ipynb`.

The core packages used in this notebook are [Mesa](https://mesa.readthedocs.io/en/stable/) to define an ABM model in Python, [EMA Workbench](https://emaworkbench.readthedocs.io/en/latest/) to design and run experiments, [SALib](https://salib.readthedocs.io/en/latest/) to conduct SA (within EMA Workbench). Also, we used one pretty plotting function of [pyNetLogo](https://pynetlogo.readthedocs.io/en/latest/).

The notebook follows a simplified SA workflow and has 5 sections-steps:

<img src="img/workflow.png"/>

## 0. Installations and imports

In [ ]:
# Clone the repo to make its file available for Google Colab
#!git clone https://github.com/BROSE-Uninc/SSF2021.git

In [ ]:
# Install necessary packages
#!pip install ema_workbench mesa ipyparallel SALib


In [1]:
# Import necessary packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import random

# Import EMA Workbench modules 
from ema_workbench import (ReplicatorModel, RealParameter, BooleanParameter, IntegerParameter, Constant, TimeSeriesOutcome, perform_experiments, save_results, ema_logging)

# Initialize logger to keep track of experiments run
ema_logging.log_to_stderr(ema_logging.INFO)

# Import covasim model
import covasim as cv  # posssibly would change this to "import covasim as cv" from covasim import Sim

ModuleNotFoundError: No module named 'ema_workbench'

## 1. Load the model

The very first step of SA with EMA Workbench is to define or "load" the model as a function. That is, EMA Workbench treats all models as functions (read *black box*). They are supposed to have **inputs** (parameters, constants, uncertainties and policy levers) and **outputs** (outcomes, KPIs). The model structure is not interesting for EMA Workbench. It may be something simple as `def func(x)` which just returns x + 1.

Our model has quite an extensive set of **8 inputs** model parameters (read more about the model [here](https://github.com/projectmesa/mesa/tree/master/examples/virus_on_network)). Their names are pretty self-explanatory, but let's quickly go through them:

1. `num_nodes` - number of network nodes,
2. `avg_node_degree` - average node degree,
3. `initial_outbreak_size` - initial number of infected nodes,
4. `virus_spread_chance` - chance of the spread of the virus,
5. `virus_check_frequency` - how often node checks,
6. `recovery_chance` - how likely the node recovers,
7. `gain_resistance_chance` - what is the chance that node get resistance,
8. `steps` - number of steps.

And **4 outputs** model outcomes which correspond to a simple SIR model:
1. `Susceptible` - number of agents in susceptible state, 
2. `Infected` - number of agents in infected state,
3. `Resistant` - number of agents in resistant state,,
4. `TIME` - to keep track of simulation time.

Now, let's parameterize and test out our Mesa model. What is supposed to happen? First, we shouldn't get any error 😅. Second, after we run `model_virus_on_network` function it has to give us a set of model outcomes. Let's try.

## 2. Design experiments

Now it's time to design experiments. What does it mean? Well, we have to specify:

* **which model parameters** aka *inputs* are we going to sample, what are their **ranges**, and random **distributions**,
* what we will keep as **constants** and do not change over the model run,
* and finally which **outcomes** we want to observe.

It's an important step in SA workflow and we have to be careful. Because if parameter ranges are too narrow or they're sampled from e.g. a Normal distribution, there is a chance that you'll overlook import model behavior. This is why model parameters are named **uncertainties** in the EMA Workbench. We often do not know parameter vales and how to explore many plausible options.

Now let's talk about "tech" part. First we have to initialize an instance of EMA Workbench called `ReplicatorModel`. This is how we "connect" EMA Workbench to our Python model. We have to pass a name of our model to `ReplicatorModel`, and also pass the function that we defined previously.

In [18]:
def model_wrapper(pars):   # creating a wrapper of the covasim simulation object
    sim = cv.Sim(pars)    # create instance of covasim simulation object called sim (putting the brackets is activating the function rather than copying it)
    return


In [19]:
model_wrapper()

TypeError: model_wrapper() missing 1 required positional argument: 'pars'

In [20]:
# Instantiate and pass the model 
model = ReplicatorModel('Sim', function=model_wrapper)

Second, let's access the model and write down its attributes. Down below you can see that we have different classes for different types of parameters: `IntegerParameter`, `RealParameter`. We use the first one for integer parameters and the second for floats. To specify a parameter we have to pass it a name, left and right boundaries for sampling. As you can see, we don't to the same for `Constants`. There you have to specify only one value. Since our model it's not static and outcomes differ over time, we have to call `TimeSeriesOutcome`. Finally we need to specify how many replications do we want to have. Rule of thumb here, let's go with 10 😎.

In [21]:
model.uncertainties = [IntegerParameter("pars.contacts.h", 2, 6)]
model.replications = 2

In [ ]:
# Define model parameters and their ranges to be sampled
model.uncertainties = [IntegerParameter("num_nodes", 10, 100),
                       IntegerParameter("avg_node_degree", 2, 8),
                       RealParameter("virus_spread_chance", 0.1, 1),
                       RealParameter("virus_check_frequency", 0.1, 1),
                       RealParameter("recovery_chance", 0.1, 1),
                       RealParameter("gain_resistance_chance", 0.1, 1)]

# Define model parameters that will remain constant
model.constants = [Constant("initial_outbreak_size", 1),
                   Constant('steps', 30)]

# Define model outcomes
model.outcomes = [TimeSeriesOutcome('TIME'),
                  TimeSeriesOutcome('Infected'),
                  TimeSeriesOutcome('Susceptible'),
                  TimeSeriesOutcome('Resistant')]

# Define the number of replications
model.replications = 10



'''

# Define model parameters and their ranges to be sampled 
# below I am choosing to vary the number of contacts at each location and the weighting to beta in each
# to also change the beta itself, that is established within 'make_pars' so not sure if it can be called in the same way
model.uncertainties = [IntegerParameter("contacts.h", 2, 6),
                       IntegerParameter("contacts.s", 6, 20),
                       IntegerParameter("contacts.w", 3,16),
                       IntegaerParameter("contacts.c", 2,20),
                       RealParameter("beta_layer.h", 0.5, 3),
                       RealParameter("beta_layer.s", 0.6, 1.5),
                       RealParameter("beta_layer.w", 0.6, 1.5),
                       RealParameter("beta_layer.c", 0.3, 1.5),
                       RealParameter("pars.beta", 0.016, 1.0)]  # this last one would be make_pars

# Define model parameters that will remain constant - the time and no. starting infections (also set in make_pars) for 365 days
model.constants = [Constant("pars.pop_infected", 20),
                   Constant('steps', 365)]

# Define model outcomes
model.outcomes = [TimeSeriesOutcome('TIME'),
                  TimeSeriesOutcome('Infected'),
                  TimeSeriesOutcome('Susceptible'),
                  TimeSeriesOutcome('Resistant')]

# Define the number of replications
model.replications = 10
    
'''


## 3. Run the model

Now, we're all set to run the model. The syntax here is pretty straightforward: we call `perform_experiments`, passing it a model instance and specify how many `scenarios` we need. Depending on the number of scenarios EMA Workbench samples the parameters. As a result, the more scenarios you pass, the more parameter combinations you get. One more attribute here is the type of `uncertainty_sampling`. Some SA methods require a specific way of how parameters should be sampled, e.g. Sobol SA. That's why we have to specify it here.

Important thing to remember: Sobol SA requires a relatively **large sample size**. As [docs](https://pynetlogo.readthedocs.io/en/latest/_docs/SALib_ipyparallel.html#Using-SALib-for-sensitivity-analysis) say, to calculate first-order, second-order and total sensitivity indices, we need to have sample size of n(2p+2), where p is the number of input parameters, and n is a baseline sample size which should be large enough to stabilize the estimation of the indices.

In [22]:
# Run experiments with the aforementioned parameters and outputs
results = perform_experiments(models=model, scenarios=2, uncertainty_sampling='sobol')

[MainProcess/INFO] performing 8 scenarios * 1 policies * 1 model(s) = 8 experiments
[MainProcess/INFO] performing experiments sequentially
[MainProcess/ERROR] model_wrapper() got an unexpected keyword argument 'pars.contacts.h'
Traceback (most recent call last):
  File "/opt/anaconda3/envs/sa-with-ema/lib/python3.8/site-packages/ema_workbench/em_framework/experiment_runner.py", line 85, in run_experiment
    model.run_model(scenario, policy)
  File "/opt/anaconda3/envs/sa-with-ema/lib/python3.8/site-packages/ema_workbench/util/ema_logging.py", line 158, in wrapper
    res = func(*args, **kwargs)
  File "/opt/anaconda3/envs/sa-with-ema/lib/python3.8/site-packages/ema_workbench/em_framework/model.py", line 306, in run_model
    output = self.run_experiment(experiment)
  File "/opt/anaconda3/envs/sa-with-ema/lib/python3.8/site-packages/ema_workbench/util/ema_logging.py", line 158, in wrapper
    res = func(*args, **kwargs)
  File "/opt/anaconda3/envs/sa-with-ema/lib/python3.8/site-package

EMAError: exception in run_model
Caused by: TypeError: model_wrapper() got an unexpected keyword argument 'pars.contacts.h'

In [ ]:
# Get the results
experiments, outcomes = results

In [ ]:
experiments.head()

In [ ]:
outcomes.keys()

Often it is a good idea to save the results of a lengthy run and open up it next day 🥱. Great that EMA Workbench has such a feature.

### Epic save

<center><img src="img/save.jpg" width=200 height=260/><center>

In [ ]:
from ema_workbench.util.utilities import save_results, load_results
import os

In [ ]:
# Create a directory to store the results
directory = 'results/virus_on_network'
if not os.path.exists(directory):
    os.makedirs(directory)

In [ ]:
# Save the results
save_results(results, directory + '/results.tar.gz')

In [ ]:
# Load the results
results = load_results(directory + '/results.tar.gz')

In [ ]:
experiments, outcomes = results

In [ ]:
outcomes.keys()

### A bit of preprocessing

Before we proceed further, we have to do a bit of preprocessing. Let's take a closer look at the outcomes. We have 4 outcomes, 1400 scenarios, 10 replications and we run the model for 30 steps. This is what you see down below:

In [ ]:
print(random.choice(list(outcomes)))
outcomes[random.choice(list(outcomes))].shape

To simplify our life 😅, let's take a mean over all replications that we had. 

In [ ]:
mean_outcomes = {key:np.mean(outcomes[key],axis=1) for key in outcomes.keys()}
mean_results = (experiments.copy(), mean_outcomes)

In [ ]:
# Now the shape of this array doesn't have 10 in it  
mean_outcomes[random.choice(list(outcomes))].shape

Note from Sophie: What I am wanting to do is see if these commands have created a means of looking at each of the mean_outcomes, for each of the experiments, in one worksheet -- has that been done? If not, I would want to do this. If I have that, it will enable me to then write some code to identify which combination of parameters produces the closest estimate to my real data. 

In [ ]:
mean_results

### Visuals!

In [ ]:
from ema_workbench.analysis.plotting import lines

In [ ]:
plt.rcParams['figure.figsize'] = [10, 12]
figure = lines(experiments, mean_outcomes)

Note from Sophie: So again here, I would import the data I have for validation, then want to find a way to compare the values over time with each of these outcomes 

## 4. Do sensitivity analysis

Finally, we're here 🧙. EMA Workbench and SALib have different methods available for sampling and SA. Here we're going to use Variance-based Sensitivity Analysis also known as Sobol (by the surname of its author). Take a look at other options available there. As usual the most complex task here is to find out whether your model fits the method. As Confucius said "*If you found a method that fits your problem within an hour, you're either a method developer or most probably bamboozled yourself.*" 
For now, let's imagine that we found the right method and it is Sobol.

The *tech* part here is relatively straightforward:

1. Specify a *problem*, or simply say what model parameters did you sample,
2. Select an *outcome* of interest, yes, we have to analyze the impact outcome by outcome,
3. Run the analysis.  

In [ ]:
from SALib.analyze import sobol
from ema_workbench.em_framework.salib_samplers import get_SALib_problem
from src.plot import plot_sobol_indices
sns.set_style('white')

In [ ]:
# Specify the problem
problem = get_SALib_problem(model.uncertainties)

In [ ]:
# Select and normalize an outcome
normalized_resistant = (mean_outcomes['Resistant'][:,-1] / experiments['num_nodes']).to_numpy()
# it has taken all the number of resistant people/number of people in that run = % of resistant individuals per run

In [ ]:
normalized_resistant  # to get the full list, do for x in normalized_resistant: print (x) 

In [ ]:
normalized_resistant.shape

In [ ]:
# Perform Sobol SA
Si = sobol.analyze(problem=problem, Y=normalized_resistant,
                   calc_second_order=True, print_to_console=False)

# which of the uncertainties (parameters that we have defined for ranges) has the most impact on the normalized_resistant outcome
# calc second order - what is the next parameter of importance (holding other things constant) like second component of PCA

# Get scores by type 
Si_filter = {k:Si[k] for k in ['ST', 'ST_conf', 'S1', 'S1_conf']}

# Create a DataFrame out of them
Si_df = pd.DataFrame(Si_filter, index=problem['names'])

# Get indices and error bars
indices = Si_df[['S1','ST']]
err = Si_df[['S1_conf','ST_conf']]

In [ ]:
# Plot the results as a barplot
fig, ax = plt.subplots(1)
indices.plot.bar(yerr=err.values.T, ax=ax)
fig.set_size_inches(8,6)
fig.subplots_adjust(bottom=0.3)

# the orange bars show what factors are the second most important (once you've considered the first)

In [ ]:
# Even a nicer plot
fig = plot_sobol_indices(Si, problem, criterion='ST', threshold=0.005)
fig.set_size_inches(7,7)

# So it's kind of like a PCA I think. 

## save as HTML

In [ ]:
!jupyter nbconvert emaworkbench_for_covasim.ipynb --to html --output emaworkbench_for_covasim_draft0.html